In [122]:
import sys
import torchio as tio
import pandas as pd 
import numpy as np
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.notebook import tqdm


sys.path.insert(0, '../../scripts/')
import data_loader
from helpers import miscellaneous as misc

CONFIG = misc.get_config()

In [11]:
CONFIG

{'RAW_DATA_DIR': 'data/raw/',
 'PROCESSED_DATA_DIR': 'data/processed/',
 'FLATTENED_DATA_DIR': 'data/raw/flattened/',
 'SERIALIZED_MODELS': 'models/trained_models/',
 'METRICS_DIR': 'models/trained_history/',
 'ANNOTATIONS_DIR': 'data/annotations/',
 'CDR_DIR': 'data/tabular_data/tabular_data/CDR.csv',
 'GDSCALE_DIR': 'data/tabular_data/tabular_data/GDSCALE.csv',
 'MMSE_DIR': 'data/tabular_data/tabular_data/MMSE.csv',
 'MOCA_DIR': 'data/tabular_data/tabular_data/MOCA.csv',
 'NEUROBAT_DIR': 'data/tabular_data/tabular_data/NEUROBAT.csv',
 'PTDEMOG_DIR': 'data/tabular_data/tabular_data/PTDEMOG.csv',
 'TRAIN_LABELS_DIR': 'data/annotations/train_labels.csv',
 'TEST_LABELS_DIR': 'data/annotations/test_labels.csv',
 'PLOT_DIR_BINARY': 'reports/plots/',
 'SLICED_DATA_DIR': 'data/raw/slices/',
 'MODEL': 'TestCNN',
 'DEVICE': 'gpu',
 'TRAIN_SIZE': 0.8,
 'LEARNING_RATE': 0.001,
 'BATCH_SIZE': 2,
 'EPOCHS': 1,
 'NUM_WORKERS': 1,
 'TRANSFORMER': 'Test',
 'CRITERION': 'CrossEntropy',
 'OPTIMIZER': '

In [83]:
SLICE = CONFIG['NSLICE']
DIMENSION = CONFIG['DIMENSION']

In [84]:
train_data = pd.read_csv('../../' + CONFIG['TRAIN_LABELS_DIR'])
test_data = pd.read_csv('../../' + CONFIG['TEST_LABELS_DIR'])

In [130]:
def imshow(img):
    plt.imshow(img)

def save_slices(annotations):

    for i in tqdm(range(len(annotations))):
        filename = annotations.filename.iloc[i]
        if isinstance(filename, str):
            name = filename.split('\\')[1]
            name = name.replace('.nii', '.png')
            
        else:
            continue
        
        img = tio.ScalarImage("../../" + filename)
        
        if SLICE == 'Center':
            nslice = img.data.size(DIMENSION)//2
        else:
            nslice = SLICE
        
        if SLICE:
            if DIMENSION == 1:
                assert img.data.size()[1] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[1]}'
                img = img.data[:, nslice, :, :]
            elif DIMENSION == 2:
                assert img.data.size()[2] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[2]}'
                img = img.data[:, :, nslice, :]
            elif DIMENSION == 3:
                assert img.data.size()[3] >= nslice, f'[WARN] Nslice is too large. Max allowed: {img.data.size()[3]}'
                img = img.data[:, :, :, nslice]

        

        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        npimg = np.squeeze(npimg)
        im = Image.fromarray(npimg).convert("L")
        im.save("../../" + CONFIG['SLICED_DATA_DIR'] + 'center/' + name)
    
    
# trainset
save_slices(train_data)
save_slices(test_data)

  0%|          | 0/1824 [00:00<?, ?it/s]

  0%|          | 0/470 [00:00<?, ?it/s]